## PLIP — Protein ligand interaction profiling

This notebook will walk us through using PLIP for identification of non-covalent interactions between biological macromolecules and their ligands.

**PLIP implementation paper**:
Adasme et al. PLIP 2021: expanding the scope of the protein-ligand interaction profiler to DNA and RNA. NAR 2021 

## 0) Setup
See [Quickstart](../index.ipynb#imports) for more details on the setup.


## 0.0) Imports

In [ ]:
import os
from pathlib import Path

import tengu
from pdbtools import pdb_fetch

## 0.1) Credentials

In [ ]:
TOKEN = os.getenv("TENGU_TOKEN")
# You might have a custom deployment url, by default the client will use https://tengu.qdx.ai
TENGU_URL = os.getenv("TENGU_URL") or "https://tengu.qdx.ai"

## 0.2) Configuration
Let's set some global variables that define our project.

In [ ]:
DESCRIPTION = "plip-inference-notebook"
TAGS = ["tengu-py", "plip", "notebook"]

WORK_DIR = Path.home() /  "qdx" / "plip-inference-tengu-py-demo"

## 0.3) Build your tengu client

In [ ]:
#|hide
if WORK_DIR.exists():
    client = tengu.Provider(workspace=WORK_DIR, access_token=TOKEN, url=TENGU_URL)
    await client.nuke(remote=True)

In [ ]:
os.makedirs(WORK_DIR, exist_ok=True)

client = await tengu.build_provider_with_functions(
    access_token=TOKEN, url=TENGU_URL, workspace=WORK_DIR, batch_tags=TAGS
)

## 0.4) Fetch example PDB
Note that PLIP requires a PDB file of a protein-ligand complex.

In [ ]:
PDB_ID = "1GIH"
FILE_NAME = f'{PDB_ID}.pdb'
FILE_PATH = WORK_DIR / FILE_NAME

In [ ]:
complex = list(pdb_fetch.fetch_structure(PDB_ID))

with open(FILE_PATH, 'w') as f:
    for line in complex:
        f.write(line)

## 1.0) Check Tengu API docs to get more information on the PLIP module
Before we call the PLIP Tengu module, we can look at the help documentation to get more information on the required inputs, expected outputs, and possible configuration parameters.


In [ ]:
help(client.plip)

Help on function plip in module tengu.provider:

async plip(*args: [<class 'pathlib.Path'>], target: tengu.graphql_client.enums.ModuleInstanceTarget | None = <ModuleInstanceTarget.NIX_SSH: 'NIX_SSH'>, resources: tengu.graphql_client.input_types.ModuleInstanceResourcesInput | None = ModuleInstanceResourcesInput(gpus=0, gpu_mem=None, gpu_mem_units=None, cpus=None, nodes=None, mem=None, mem_units=None, storage=138, storage_units=<MemUnits.MB: 'MB'>, walltime=None, storage_mounts=None), tags: list[str] | None = None, restore: bool | None = None) -> [<class 'pathlib.Path'>, <class 'pathlib.Path'>]
    Run PLIP on the input PDB file of a protein-ligand complex to obtain interaction data.
    
    
    Module version: github:talo/tengu-plip/9a4ff5b9e5fe77f37505c4718d0d63182885c805#plip_tengu
    
    QDX Type Description:
    
        input_complex_pdb_file: @bytes 
    
    ->
    
        output_conformer: @Conformer;
    
        output_txt_file: @bytes
    
    
    
    :param input_comp

We can see from the above help documentation that we need to pass the Path to our PDB file as an argument.

## 1.1) Run PLIP
Finally, we run PLIP so we can identify interactions between the ligand and the protein within our protein complex.

In [ ]:
PLIP_RESOURCES = {
    'storage':  1024_000
}

output_conformer, output_txt = await client.plip(FILE_PATH, resources=PLIP_RESOURCES)

## 1.2) Get output conformer
Here, we get the output Conformer (in QDXF format) that is enriched with `binding_site_interactions` by PLIP.

In [ ]:
output_conformer = await output_conformer.get()

2024-01-23 17:19:14,956 - tengu - INFO - Argument 9dff140f-714f-4391-8dd6-70205c9b50e8 is now ModuleInstanceStatus.RESOLVING
2024-01-23 17:19:38,365 - tengu - INFO - Argument 9dff140f-714f-4391-8dd6-70205c9b50e8 is now ModuleInstanceStatus.ADMITTED
2024-01-23 17:19:52,036 - tengu - INFO - Argument 9dff140f-714f-4391-8dd6-70205c9b50e8 is now ModuleInstanceStatus.DISPATCHED
2024-01-23 17:19:57,573 - tengu - INFO - Argument 9dff140f-714f-4391-8dd6-70205c9b50e8 is now ModuleInstanceStatus.QUEUED


## 1.3) Get PLIP output document
PLIP also returns a link to the output document of the results of the PLIP output.

In [ ]:
output_txt = await output_txt.get()